# Advanced Likelihood: Summed Likelihood


## Summed Likelihood Analysis
The summed likelihood method available in the Fermi Science tools is a way of performing a joint likelihood fit on two data selections using the same XML model, i.e., you can estimate a single parameter simultaneously for multiple data sets. Remember this is exactly what the likelihood formalism implies. You can't stop me from writing down a likelihood function containing as many independent data sets as I like as long as I have a response matrix for each selection. 

This is useful if you want to do the following:

* Coanalysis of Front and Back selections (not using the combined IRF for whatever reason)
* Coanalysis of separate time intervals
* Coanalysis of separate energy ranges
* Pass 8 PSF type analysis
* Pass 8 EDISP type analysis

There are many other possible applications. Remember that the likelihood framework is very flexible! You're mainly limited by your imagination.

### Example with 3C 279
We're going to use the 3C 279 data set that we know and love and perform a seperate front/back analysis and then perform a summed likelihood at the end.  We're going to do a binned anlysis this time since we haven't really done one yet.

I've provided a lot of the ancillary files in a tarball linked to the agenda.  Let's look at the directory

In [1]:
!ls -1

3C279_back_filtered.fits
3C279_front_filtered.fits
events.txt
L1405221252264C652E7F67_PH00.fits
L1405221252264C652E7F67_PH01.fits
L1405221252264C652E7F67_SC00.fits
Summed_and_Composite.ipynb


Note that you need to place the galactic diffuse model and the raw data files in the working directory (gll_iem_v05_rev1.fit, L1405221252264C652E7F67_PH00.fits and L1405221252264C652E7F67_SC00.fits).  I didn't put them in the tar ball since they are big and you already have them in your VM.  You'll need to copy or link isotropic models for the front and back data selections into your working directory (iso_source_back_v05_rev1.fits and iso_source_front_v05_rev1.fits) -- check.  These two files are in $FERMI_DIR/refdata/fermi/galdiffuse. 

Now, we need to import some functions so that we can work on these data.  Note that you don't have to run all of these.  If I say 'Don't run this', don't run it; the output file is already generated for you.

In [2]:
from gt_apps import filter, maketime, expMap, expCube, evtbin, srcMaps

#### Selecting Front and Back Events

**<font color='red'>Don't run these commands</font>** (unless you want to rerun everything).

We need to run gtselect (called 'filter' in python) twice.  Once, we select only the front events and the other time we select only back events.  You do this with the hidden parameter 'convtype'.

In [3]:
filter['rad'] = 15
filter['evclass'] = 2
filter['infile'] = '@events.txt'
filter['outfile'] = '3C279_front_filtered.fits'
filter['ra'] = 194.046527
filter['dec'] = -5.789312
filter['tmin'] = 239557417
filter['tmax'] = 255398400
filter['emin'] = 100
filter['emax'] = 100000
filter['zmax'] = 100
filter['convtype'] = 0

In [4]:
filter.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtselect infile=@events.txt outfile=3C279_front_filtered.fits ra=194.046527 dec=-5.789312 rad=15.0 tmin=239557417.0 tmax=255398400.0 emin=100.0 emax=100000.0 zmax=100.0 evclsmin="INDEF" evclsmax="INDEF" evclass=2 convtype=0 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"
Done.
real 1.89
user 0.11
sys 1.11


In [5]:
filter['rad'] = 15
filter['evclass'] = 2
filter['infile'] = "@events.txt"
filter['outfile'] = "3C279_back_filtered.fits"
filter['ra'] = 194.046527
filter['dec'] = -5.789312
filter['tmin'] = 239557417
filter['tmax'] = 255398400
filter['emin'] = 100
filter['emax'] = 100000
filter['zmax'] = 100
filter['convtype'] = 1

In [6]:
filter.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtselect infile=@events.txt outfile=3C279_back_filtered.fits ra=194.046527 dec=-5.789312 rad=15.0 tmin=239557417.0 tmax=255398400.0 emin=100.0 emax=100000.0 zmax=100.0 evclsmin="INDEF" evclsmax="INDEF" evclass=2 convtype=1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"
Done.
real 1.97
user 0.09
sys 1.22


#### Compute the GTIs

**<font color='red'>Don't run these commands.</font>** (unless you want to rerun everything).

Now, we need to find the GTIs for each data set (front and back).

In [7]:
maketime['scfile'] = 'L1405221252264C652E7F67_SC00.fits'
maketime['filter'] = '(DATA_QUAL==1)&&(LAT_CONFIG==1)'
maketime['roicut'] = 'yes'
maketime['evfile'] = '3C279_front_filtered.fits'
maketime['outfile'] = '3C279_front_filtered_gti.fits'

In [8]:
maketime.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtmktime scfile=L1405221252264C652E7F67_SC00.fits sctable="SC_DATA" filter="(DATA_QUAL==1)&&(LAT_CONFIG==1)" roicut=yes evfile=3C279_front_filtered.fits evtable="EVENTS" outfile="3C279_front_filtered_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"
real 3.36
user 0.66
sys 1.28


In [9]:
maketime['scfile'] = 'L1405221252264C652E7F67_SC00.fits'
maketime['filter'] = '(DATA_QUAL==1)&&(LAT_CONFIG==1)'
maketime['roicut'] = 'yes'
maketime['evfile'] = '3C279_back_filtered.fits'
maketime['outfile'] = '3C279_back_filtered_gti.fits'

In [10]:
maketime.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtmktime scfile=L1405221252264C652E7F67_SC00.fits sctable="SC_DATA" filter="(DATA_QUAL==1)&&(LAT_CONFIG==1)" roicut=yes evfile=3C279_back_filtered.fits evtable="EVENTS" outfile="3C279_back_filtered_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"
real 2.82
user 0.48
sys 1.50


#### Compute the Livetime Cube

You only need to do this once sincein this case we made the exact same time cuts and used the same GTI filter on both data sets.

**<font color='red'>Don't run these commands.</font>** (unless you want to rerun everything).


In [11]:
expCube['evfile'] = '3C279_front_filtered_gti.fits'
expCube['scfile'] = 'L1405221252264C652E7F67_SC00.fits'
expCube['outfile'] = '3C279_front_ltcube.fits'
expCube['dcostheta'] = 0.025
expCube['binsz'] = 1.0

In [12]:
expCube.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtltcube evfile="3C279_front_filtered_gti.fits" evtable="EVENTS" scfile=L1405221252264C652E7F67_SC00.fits sctable="SC_DATA" outfile=3C279_front_ltcube.fits dcostheta=0.025 binsz=1.0 phibins=0 tmin=0.0 tmax=0.0 file_version="1" zmin=0.0 zmax=180.0 chatter=2 clobber=yes debug=no gui=no mode="ql"
Working on file L1405221252264C652E7F67_SC00.fits
.....................!
real 574.72
user 547.75
sys 6.94


#### Compute the counts cube

**<font color='red'>Don't run these commands.</font>** (unless you want to rerun everything).

The counts cube is the counts from our data file binned in space and energy.  All of the steps above use a circular ROI (or a cone, really).  Once you switch to binned analysis, you start doing things in squares.  Your counts cube can only be as big as the biggest square that can fit in the circular ROI you already selected.  

![CCUBE Geometry](http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/images/BinnedLikelihood/square_in_circle.png)

This basically means that width of your square, s, should be less than $r\times\sqrt(2)$.  We are using an ROI with a radius, r, of 15, so we can only use a square with sides of size:

In [13]:
import numpy as np
15.*np.sqrt(2)

21.213203435596427

So, we're going to make a square with 100 pixels in each dimension and set the degrees/pixel to 0.2 so that we have a 20x20 degree cube.  We are also making 30 energy bins with logarithmic spacing.  You have to bin up both the front and back data sets.

In [14]:
evtbin['evfile'] = '3C279_front_filtered_gti.fits'
evtbin['outfile'] = '3C279_front_CCUBE.fits'
evtbin['algorithm'] = 'CCUBE'
evtbin['nxpix'] = 100
evtbin['nypix'] = 100
evtbin['binsz'] = 0.2
evtbin['coordsys'] = 'CEL'
evtbin['xref'] = 194.046527
evtbin['yref'] =  -5.789312
evtbin['axisrot'] = 0
evtbin['proj'] = 'AIT'
evtbin['ebinalg'] = 'LOG'
evtbin['emin'] = 100
evtbin['emax'] = 100000
evtbin['enumbins'] = 30
evtbin['scfile'] = 'L1405221252264C652E7F67_SC00.fits'

In [15]:
evtbin.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtbin evfile=3C279_front_filtered_gti.fits scfile=L1405221252264C652E7F67_SC00.fits outfile=3C279_front_CCUBE.fits algorithm="CCUBE" ebinalg="LOG" emin=100.0 emax=100000.0 enumbins=30 ebinfile=NONE tbinalg="LIN" tbinfile=NONE nxpix=100 nypix=100 binsz=0.2 coordsys="CEL" xref=194.046527 yref=-5.789312 axisrot=0.0 rafield="RA" decfield="DEC" proj="AIT" hpx_ordering_scheme="RING" hpx_order=3 hpx_ebin=yes evtable="EVENTS" sctable="SC_DATA" efield="ENERGY" tfield="TIME" chatter=2 clobber=yes debug=no gui=no mode="ql"
This is gtbin version ScienceTools-v9r33p0-fssc-20140520
real 1.52
user 0.05
sys 1.05


In [16]:
evtbin['evfile'] = '3C279_back_filtered_gti.fits'
evtbin['outfile'] = '3C279_back_CCUBE.fits'
evtbin['algorithm'] = 'CCUBE'
evtbin['nxpix'] = 100
evtbin['nypix'] = 100
evtbin['binsz'] = 0.2
evtbin['coordsys'] = 'CEL'
evtbin['xref'] = 194.046527
evtbin['yref'] =  -5.789312
evtbin['axisrot'] = 0
evtbin['proj'] = 'AIT'
evtbin['ebinalg'] = 'LOG'
evtbin['emin'] = 100
evtbin['emax'] = 100000
evtbin['enumbins'] = 30
evtbin['scfile'] = 'L1405221252264C652E7F67_SC00.fits'

In [17]:
evtbin.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtbin evfile=3C279_back_filtered_gti.fits scfile=L1405221252264C652E7F67_SC00.fits outfile=3C279_back_CCUBE.fits algorithm="CCUBE" ebinalg="LOG" emin=100.0 emax=100000.0 enumbins=30 ebinfile=NONE tbinalg="LIN" tbinfile=NONE nxpix=100 nypix=100 binsz=0.2 coordsys="CEL" xref=194.046527 yref=-5.789312 axisrot=0.0 rafield="RA" decfield="DEC" proj="AIT" hpx_ordering_scheme="RING" hpx_order=3 hpx_ebin=yes evtable="EVENTS" sctable="SC_DATA" efield="ENERGY" tfield="TIME" chatter=2 clobber=yes debug=no gui=no mode="ql"
This is gtbin version ScienceTools-v9r33p0-fssc-20140520
real 1.55
user 0.05
sys 1.07


#### Generate a Binned Exposure Map

The binned exposure map is an exposure map binned in space and energy.  

**<font color='red'>Don't run these commands.</font>** (unless you want to rerun everything).


You first need to import the python version of 'gtexpcube2' which doesn't have a gtapp version by default.  It's easy to do (you can import any of the command line tools into python this way).  Then, you can check out the parameters with the pars function.

In [18]:
from GtApp import GtApp
expCube2 = GtApp('gtexpcube2','Likelihood')

In [19]:
expCube2.pars()

' infile= cmap= outfile= irfs="CALDB" nxpix=360 nypix=180 binsz=1.0 coordsys="GAL" xref=0.0 yref=0.0 axisrot=0.0 proj="CAR" ebinalg="LOG" emin=100.0 emax=300000.0 enumbins=10 ebinfile="NONE" bincalc="EDGE" ignorephi=no thmax=180.0 thmin=0.0 table="EXPOSURE" chatter=2 clobber=yes debug=no mode="ql"'

There's a tricky bit here that came up when we discussed binned exposure maps.  You need to compute the exposure beyond the spatial bounds of your counts cube.  This is because in your likelihood model you need consider photons produced by a source lying outside your ROI but within range of the PSF.  There is no harm in generating a larger exposure map than you need. You could generate one for the whole sky to cover all possible regions. Since this does increase the calculation time, you might prefer something a little smaller. 

A good rule of thumb is to add a margin to the exposure cube spatial size that is larger than the 68% value for the PSF at the lowest energy in your analysis.  At 100 MeV the LAT has a PSF of about 10 degrees, thus, we need to add more than 10 degrees to both sides of our square, or make an exposure cube with a side length of more than 20+10+10 = 40 deg. In this case, to be safe we'll make the exposure map using 300 pixels x 300 pixels with the same pixel size (0.2 deg) that we used for the counts cube. The result is an exposure cube that is 60 deg x 60 deg and leaves plenty of margin beyond a 20 deg x 20 deg counts cube.  For the energy binning, use the same size as the counts cube.

The exposure calculation requires an IRF set. For each data selection, you should use the IRF that matches the data, and so here you should specify the front or back IRFs as appropriate.

In [20]:
expCube2['infile'] = '3C279_front_ltcube.fits'
expCube2['cmap'] = 'none'
expCube2['outfile'] = '3C279_front_BinnedExpMap.fits'
expCube2['irfs'] = 'P7REP_SOURCE_V15::FRONT'
expCube2['nxpix'] = 300
expCube2['nypix'] = 300
expCube2['binsz'] = 0.2
expCube2['coordsys'] = 'CEL'
expCube2['xref'] = 194.046527
expCube2['yref'] = -5.789312
expCube2['axisrot'] = 0.0
expCube2['proj'] = 'AIT'
expCube2['ebinalg'] = 'LOG'
expCube2['emin'] = 100
expCube2['emax'] = 100000
expCube2['enumbins'] = 30

In [21]:
expCube2.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtexpcube2 infile=3C279_front_ltcube.fits cmap=none outfile=3C279_front_BinnedExpMap.fits irfs="P7REP_SOURCE_V15::FRONT" nxpix=300 nypix=300 binsz=0.2 coordsys="CEL" xref=194.046527 yref=-5.789312 axisrot=0.0 proj="AIT" ebinalg="LOG" emin=100.0 emax=100000.0 enumbins=30 ebinfile="NONE" bincalc="EDGE" ignorephi=no thmax=180.0 thmin=0.0 table="EXPOSURE" chatter=2 clobber=yes debug=no mode="ql"
Computing binned exposure map....................!
real 8.99
user 4.97
sys 2.55


In [22]:
expCube2['infile'] = '3C279_front_ltcube.fits'
expCube2['cmap'] = 'none'
expCube2['outfile'] = '3C279_back_BinnedExpMap.fits'
expCube2['irfs'] = 'P7REP_SOURCE_V15::BACK'
expCube2['nxpix'] = 300
expCube2['nypix'] = 300
expCube2['binsz'] = 0.2
expCube2['coordsys'] = 'CEL'
expCube2['xref'] = 194.046527
expCube2['yref'] = -5.789312
expCube2['axisrot'] = 0.0
expCube2['proj'] = 'AIT'
expCube2['ebinalg'] = 'LOG'
expCube2['emin'] = 100
expCube2['emax'] = 100000
expCube2['enumbins'] = 30

In [23]:
expCube2.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtexpcube2 infile=3C279_front_ltcube.fits cmap=none outfile=3C279_back_BinnedExpMap.fits irfs="P7REP_SOURCE_V15::BACK" nxpix=300 nypix=300 binsz=0.2 coordsys="CEL" xref=194.046527 yref=-5.789312 axisrot=0.0 proj="AIT" ebinalg="LOG" emin=100.0 emax=100000.0 enumbins=30 ebinfile="NONE" bincalc="EDGE" ignorephi=no thmax=180.0 thmin=0.0 table="EXPOSURE" chatter=2 clobber=yes debug=no mode="ql"
Computing binned exposure map....................!
real 8.63
user 4.95
sys 2.49


#### Compute the sourcemaps

The sourcemaps step convolves the LAT response with your source model generating maps for each source in the model for use in the likelihood calculation.

**<font color='red'>Don't run these commands.</font>** (unless you want to rerun everything).

In [24]:
srcMaps['scfile'] = 'L1405221252264C652E7F67_SC00.fits'
srcMaps['expcube'] = '3C279_front_ltcube.fits'
srcMaps['cmap'] = '3C279_front_CCUBE.fits'
srcMaps['srcmdl'] = '3C279_input_model_front.xml'
srcMaps['bexpmap'] = '3C279_front_BinnedExpMap.fits'
srcMaps['outfile'] = '3C279_front_srcMap.fits'
srcMaps['irfs'] = 'P7REP_SOURCE_V15::FRONT'

In [29]:
srcMaps.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtsrcmaps scfile=L1405221252264C652E7F67_SC00.fits sctable="SC_DATA" expcube=3C279_front_ltcube.fits cmap=3C279_front_CCUBE.fits srcmdl=3C279_input_model_front.xml bexpmap=3C279_front_BinnedExpMap.fits outfile=3C279_front_srcMap.fits irfs="P7REP_SOURCE_V15::FRONT" convol=yes resample=yes rfactor=2 minbinsz=0.1 ptsrc=yes psfcorr=yes emapbnds=yes copyall=no chatter=2 clobber=yes debug=no gui=no mode="ql"
Generating SourceMap for 3C 273....................!
Generating SourceMap for 3C 279....................!
Generating SourceMap for gll_iem_v05_rev1....................!
Generating SourceMap for iso_source_v05....................!
real 108.95
user 97.15
sys 8.08


In [30]:
srcMaps['scfile'] = 'L1405221252264C652E7F67_SC00.fits'
srcMaps['expcube'] = '3C279_front_ltcube.fits'
srcMaps['cmap'] = '3C279_back_CCUBE.fits'
srcMaps['srcmdl'] = '3C279_input_model_back.xml'
srcMaps['bexpmap'] = '3C279_back_BinnedExpMap.fits'
srcMaps['outfile'] = '3C279_back_srcMap.fits'
srcMaps['irfs'] = 'P7REP_SOURCE_V15::BACK'

In [31]:
srcMaps.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtsrcmaps scfile=L1405221252264C652E7F67_SC00.fits sctable="SC_DATA" expcube=3C279_front_ltcube.fits cmap=3C279_back_CCUBE.fits srcmdl=3C279_input_model_back.xml bexpmap=3C279_back_BinnedExpMap.fits outfile=3C279_back_srcMap.fits irfs="P7REP_SOURCE_V15::BACK" convol=yes resample=yes rfactor=2 minbinsz=0.1 ptsrc=yes psfcorr=yes emapbnds=yes copyall=no chatter=2 clobber=yes debug=no gui=no mode="ql"
Generating SourceMap for 3C 273....................!
Generating SourceMap for 3C 279....................!
Generating SourceMap for gll_iem_v05_rev1....................!
Generating SourceMap for iso_source_v05....................!
real 108.52
user 97.72
sys 5.99


#### Now, Perform the Likelihood Analysis

**<font color='red'>Start running commands now if you want.</font>**

First, import the BinnedAnalysis and SummedAnalysis libraries.

In [32]:
from BinnedAnalysis import *
from SummedLikelihood import *

Then, create a likelihood object for both the front and back data sets.

In [33]:
like_f = binnedAnalysis(irfs='P7REP_SOURCE_V15::FRONT', 
                        expcube='3C279_front_ltcube.fits', 
                        srcmdl='3C279_input_model_front.xml',
                        optimizer='NEWMINUIT',
                        cmap='3C279_front_srcMap.fits',
                        bexpmap='3C279_front_BinnedExpMap.fits')

like_b = binnedAnalysis(irfs='P7REP_SOURCE_V15::BACK', 
                        expcube='3C279_front_ltcube.fits', 
                        srcmdl='3C279_input_model_back.xml',
                        optimizer='NEWMINUIT',
                        cmap='3C279_back_srcMap.fits',
                        bexpmap='3C279_back_BinnedExpMap.fits')

Then, create the summed likelihood object and add the two likelihood objects, one for the front selection and the second for the back selection.

In [34]:
summed_like = SummedLikelihood()
summed_like.addComponent(like_f)
summed_like.addComponent(like_b)

Perform the fit and print out the results.

In [35]:
summed_like.fit(0)

121893.38576423892

In [36]:
summed_like.model

3C 273
   Spectrum: PowerLaw
0      Prefactor:  9.342e+00  3.351e-01  1.000e-03  1.000e+03 ( 1.000e-09)
1          Index: -2.639e+00  2.587e-02 -5.000e+00 -1.000e+00 ( 1.000e+00)
2          Scale:  1.000e+02  0.000e+00  3.000e+01  2.000e+03 ( 1.000e+00) fixed

3C 279
   Spectrum: PowerLaw
3      Prefactor:  8.173e+00  2.554e-01  1.000e-03  1.000e+03 ( 1.000e-09)
4          Index: -2.236e+00  1.695e-02 -5.000e+00 -1.000e+00 ( 1.000e+00)
5          Scale:  1.000e+02  0.000e+00  3.000e+01  2.000e+03 ( 1.000e+00) fixed

gll_iem_v05_rev1
   Spectrum: ConstantValue
6          Value:  1.398e+00  2.670e-02  0.000e+00  1.000e+01 ( 1.000e+00)

iso_source_v05
   Spectrum: FileFunction
7     Normalization:  1.147e+00  2.984e-02  1.000e-05  1.000e+03 ( 1.000e+00)

In [38]:
summed_like.Ts('3C 279')

9797.14946402298

In [40]:
summed_like.NpredValue('3C 279')

5681.121429178282

#### Summary

Now, we have maximized the likelihood for our model simultaneously for two data sets with distinct IRFs, the front and back event type selections. Let's compare with the standard analysis that uses only one data set containing both Front and Back event types that are represented by a single, combined IRF set. Since we did not run the binned analysis for the combined front + back events yet, I'll do that below.

The TS for the front + back analysis is 9357, a bit lower than what we found for the separate front and back analysis. The important difference is that in the separated version of the analysis each event type has a dedicated response function set instead of using the averaged Front+Back response. This should increase the sensitivity, and therefore, the TS value. 

Note that the TS value for the binned analysis is lower than what we found for the unbinned likelihood analysis of the same data. That could be because the binning chosen for our analysis does not preserve all of the response information for each photon, meaning the spatial binning may be too course for some energy bins. We could test that by rerunning the binned analysis with a finer spatial binning scheme. One nice application of summed likelihood is to combine a binned analysis below 1 GeV or so, where the spatial information is less precise and the number of events is much higher, and to run an unbinned analysis above 1 GeV, where the data have higher spatial resolution but are more sparse.

#### Standard Front + Back Analysis for Comparison

In [41]:
filter['rad'] = 15
filter['evclass'] = 2
filter['infile'] = '@events.txt'
filter['outfile'] = '3C279_filtered.fits'
filter['ra'] = 194.046527
filter['dec'] = -5.789312
filter['tmin'] = 239557417
filter['tmax'] = 255398400
filter['emin'] = 100
filter['emax'] = 100000
filter['zmax'] = 100
filter['convtype'] = -1

In [42]:
filter.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtselect infile=@events.txt outfile=3C279_filtered.fits ra=194.046527 dec=-5.789312 rad=15.0 tmin=239557417.0 tmax=255398400.0 emin=100.0 emax=100000.0 zmax=100.0 evclsmin="INDEF" evclsmax="INDEF" evclass=2 convtype=-1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"
Done.
real 2.51
user 0.13
sys 1.53


In [44]:
maketime['scfile'] = 'L1405221252264C652E7F67_SC00.fits'
maketime['filter'] = '(DATA_QUAL==1)&&(LAT_CONFIG==1)'
maketime['roicut'] = 'yes'
maketime['evfile'] = '3C279_filtered.fits'
maketime['outfile'] = '3C279_filtered_gti.fits'

In [45]:
maketime.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtmktime scfile=L1405221252264C652E7F67_SC00.fits sctable="SC_DATA" filter="(DATA_QUAL==1)&&(LAT_CONFIG==1)" roicut=yes evfile=3C279_filtered.fits evtable="EVENTS" outfile="3C279_filtered_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"
real 3.15
user 0.61
sys 1.49


In [49]:
evtbin['evfile'] = '3C279_filtered_gti.fits'
evtbin['outfile'] = '3C279_CCUBE.fits'
evtbin['algorithm'] = 'CCUBE'
evtbin['nxpix'] = 100
evtbin['nypix'] = 100
evtbin['binsz'] = 0.2
evtbin['coordsys'] = 'CEL'
evtbin['xref'] = 194.046527
evtbin['yref'] =  -5.789312
evtbin['axisrot'] = 0
evtbin['proj'] = 'AIT'
evtbin['ebinalg'] = 'LOG'
evtbin['emin'] = 100
evtbin['emax'] = 100000
evtbin['enumbins'] = 30
evtbin['scfile'] = 'L1405221252264C652E7F67_SC00.fits'

In [50]:
evtbin.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtbin evfile=3C279_filtered_gti.fits scfile=L1405221252264C652E7F67_SC00.fits outfile=3C279_CCUBE.fits algorithm="CCUBE" ebinalg="LOG" emin=100.0 emax=100000.0 enumbins=30 ebinfile=NONE tbinalg="LIN" tbinfile=NONE nxpix=100 nypix=100 binsz=0.2 coordsys="CEL" xref=194.046527 yref=-5.789312 axisrot=0.0 rafield="RA" decfield="DEC" proj="AIT" hpx_ordering_scheme="RING" hpx_order=3 hpx_ebin=yes evtable="EVENTS" sctable="SC_DATA" efield="ENERGY" tfield="TIME" chatter=2 clobber=yes debug=no gui=no mode="ql"
This is gtbin version ScienceTools-v9r33p0-fssc-20140520
real 1.65
user 0.05
sys 1.14


In [46]:
expCube2['infile'] = '3C279_front_ltcube.fits'
expCube2['cmap'] = 'none'
expCube2['outfile'] = '3C279_BinnedExpMap.fits'
expCube2['irfs'] = 'P7REP_SOURCE_V15'
expCube2['nxpix'] = 300
expCube2['nypix'] = 300
expCube2['binsz'] = 0.2
expCube2['coordsys'] = 'CEL'
expCube2['xref'] = 194.046527
expCube2['yref'] = -5.789312
expCube2['axisrot'] = 0.0
expCube2['proj'] = 'AIT'
expCube2['ebinalg'] = 'LOG'
expCube2['emin'] = 100
expCube2['emax'] = 100000
expCube2['enumbins'] = 30

In [48]:
expCube2.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtexpcube2 infile=3C279_front_ltcube.fits cmap=none outfile=3C279_BinnedExpMap.fits irfs="P7REP_SOURCE_V15" nxpix=300 nypix=300 binsz=0.2 coordsys="CEL" xref=194.046527 yref=-5.789312 axisrot=0.0 proj="AIT" ebinalg="LOG" emin=100.0 emax=100000.0 enumbins=30 ebinfile="NONE" bincalc="EDGE" ignorephi=no thmax=180.0 thmin=0.0 table="EXPOSURE" chatter=2 clobber=yes debug=no mode="ql"
Computing binned exposure map....................!
real 14.51
user 10.69
sys 2.33


In [51]:
srcMaps['scfile'] = 'L1405221252264C652E7F67_SC00.fits'
srcMaps['expcube'] = '3C279_front_ltcube.fits'
srcMaps['cmap'] = '3C279_CCUBE.fits'
srcMaps['srcmdl'] = '3C279_input_model.xml'
srcMaps['bexpmap'] = '3C279_BinnedExpMap.fits'
srcMaps['outfile'] = '3C279_srcMap.fits'
srcMaps['irfs'] = 'P7REP_SOURCE_V15'

In [54]:
srcMaps.run()

time -p /home/fermiuser/AstroSoft/ScienceTools/x86_64-unknown-linux-gnu-libc2.12/bin/gtsrcmaps scfile=L1405221252264C652E7F67_SC00.fits sctable="SC_DATA" expcube=3C279_front_ltcube.fits cmap=3C279_CCUBE.fits srcmdl=3C279_input_model.xml bexpmap=3C279_BinnedExpMap.fits outfile=3C279_srcMap.fits irfs="P7REP_SOURCE_V15" convol=yes resample=yes rfactor=2 minbinsz=0.1 ptsrc=yes psfcorr=yes emapbnds=yes copyall=no chatter=2 clobber=yes debug=no gui=no mode="ql"
Generating SourceMap for 3C 273....................!
Generating SourceMap for 3C 279....................!
Generating SourceMap for gll_iem_v05_rev1....................!
Generating SourceMap for iso_source_v05....................!
real 112.46
user 99.31
sys 7.05


In [55]:
like = binnedAnalysis(irfs='P7REP_SOURCE_V15', 
                        expcube='3C279_front_ltcube.fits', 
                        srcmdl='3C279_input_model.xml',
                        optimizer='NEWMINUIT',
                        cmap='3C279_srcMap.fits',
                        bexpmap='3C279_BinnedExpMap.fits')

In [56]:
like.fit(0)

89892.82922313765

In [58]:
like.model

3C 273
   Spectrum: PowerLaw
0      Prefactor:  9.187e+00  3.344e-01  1.000e-03  1.000e+03 ( 1.000e-09)
1          Index: -2.629e+00  2.589e-02 -5.000e+00 -1.000e+00 ( 1.000e+00)
2          Scale:  1.000e+02  0.000e+00  3.000e+01  2.000e+03 ( 1.000e+00) fixed

3C 279
   Spectrum: PowerLaw
3      Prefactor:  8.010e+00  2.536e-01  1.000e-03  1.000e+03 ( 1.000e-09)
4          Index: -2.227e+00  1.698e-02 -5.000e+00 -1.000e+00 ( 1.000e+00)
5          Scale:  1.000e+02  0.000e+00  3.000e+01  2.000e+03 ( 1.000e+00) fixed

gll_iem_v05_rev1
   Spectrum: ConstantValue
6          Value:  1.365e+00  2.810e-02  0.000e+00  1.000e+01 ( 1.000e+00)

iso_source_v05
   Spectrum: FileFunction
7     Normalization:  1.188e+00  3.165e-02  1.000e-05  1.000e+03 ( 1.000e+00)

In [60]:
like.Ts('3C 279')

9357.415123912418

In [61]:
like.NpredValue('3C 279')

5629.9876856669225